In [1]:
import pandas as pd
import time as time
import requests

In [2]:
# This function scrapes tot_posts from Reddit including [removed] and [deleted] posts

def scrape_reddit(subreddit, tot_posts):
    
    url = 'https://api.pushshift.io/reddit/search/submission'
    df = pd.DataFrame()
    posts_scraped = 0

    while len(df) < tot_posts:
        if tot_posts < 100:
            params = {
                'subreddit': subreddit,
                'size': tot_posts,
                'before': int(time.time())
            }
            res = requests.get(url, params)
            data = res.json()
            posts = data['data']
            df = pd.DataFrame(posts)
        else:
            if len(df) == 0:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': int(time.time())
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df = pd.DataFrame(posts)
                posts_scraped += 100
            elif tot_posts - posts_scraped > 100:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': df.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
                posts_scraped += 100
            else:
                params = {
                    'subreddit': subreddit,
                    'size': (tot_posts - posts_scraped),
                    'before': df.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
        
    return df

In [3]:
ask_men = scrape_reddit('AskMen', 1000)

In [4]:
len(ask_men)

1000

In [5]:
ask_men[['subreddit', 'id', 'title', 'selftext', 'domain']].head(25)

,subreddit,id,title,selftext,domain
0,AskMen,j9jhlq,How do I be a man at 18? And how do I stop bei...,"I’m 18, all my life I’ve been called and told ...",self.AskMen
1,AskMen,j9jhbl,Is it okay to masturbate before the testicle c...,[removed],self.AskMen
2,AskMen,j9jeu4,What has a female done to make you feel wanted...,,self.AskMen
3,AskMen,j9jdk1,Rogaine (minoxidil) and Blood thinners,NaN,self.AskMen
4,AskMen,j9jcm6,How can you meet women during the pandemic wit...,Curious what success stories or other info my ...,self.AskMen
5,AskMen,j9j0w5,"Whats the best in condom in terms of strength,...",,self.AskMen
6,AskMen,j9iyzo,"Men who are physical abusive in relationships,...",,self.AskMen
7,AskMen,j9iyte,How can I help with panic attacks?,"Obligatory, I’m a female.\n\nI’ve been single ...",self.AskMen
8,AskMen,j9ivpy,How do I feel a little more human?,Lately I’ve found myself sitting on the couch ...,self.AskMen
9,AskMen,j9isld,Any advice on how to emotionally deal with bes...,It just happened so fast and so surreal. This ...,self.AskMen


In [6]:
# https://stackoverflow.com/questions/20995196/python-pandas-counting-and-summing-specific-conditions

# ask_men[ask_men.selftext != '[removed]'].sum()   

len(ask_men[(ask_men.selftext != '[removed]') & (ask_men.selftext != '[deleted]')])

440

In [13]:
# This function scrapes at least min_posts from Reddit not including [removed] and [deleted] posts

def scrape_reddit(subreddit, min_posts):
     
    url = 'https://api.pushshift.io/reddit/search/submission'
    df = pd.DataFrame()
    posts_scraped = 0

    while len(df) < min_posts:
#         print('len =', len(df))
        if len(df) == 0:
            params = {
                'subreddit': subreddit,
                'size': 100,
                'before': int(time.time())
            }
            res = requests.get(url, params)
            data = res.json()
            posts = data['data']
            df = pd.DataFrame(posts)
            df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
            df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
            
        else:
            params = {
                'subreddit': subreddit,
                'size': 100,
                'before': df.iloc[-1]['created_utc']
            }
            res = requests.get(url, params)
            data = res.json()
            posts = data['data']
            df2 = pd.DataFrame(posts)
            df = df.append(df2, ignore_index=True)
            df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
            df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
        
    return df

In [14]:
ask_men = scrape_reddit('AskMen', 300)

In [15]:
ask_men.shape

(313, 64)

In [17]:
ask_men[['subreddit', 'id', 'title', 'selftext', 'domain']].head(25)

,subreddit,id,title,selftext,domain
0,AskMen,j9jszp,How do you deal with a bad haircut?,The barber just completely ended me and I don’...,self.AskMen
1,AskMen,j9jsz1,What's your reaction when you finally see a fe...,,self.AskMen
2,AskMen,j9jr1t,How do I get some direction in my life at 23?,So I am going to be 23 this year and I have be...,self.AskMen
3,AskMen,j9jo4o,How important is respect and are you willing t...,,self.AskMen
4,AskMen,j9jnfl,How do you overcome body insecurity?,Not a day goes by where I don't cringe at my s...,self.AskMen
5,AskMen,j9jnbq,"How important is respect, how do you earn it, ...",,self.AskMen
6,AskMen,j9jn5d,Why will there always be a need for infantry? ...,,self.AskMen
7,AskMen,j9jm11,How do you know if a girl wants a dick pic?,,self.AskMen
8,AskMen,j9jhlq,How do I be a man at 18? And how do I stop bei...,"I’m 18, all my life I’ve been called and told ...",self.AskMen
9,AskMen,j9jeu4,What has a female done to make you feel wanted...,,self.AskMen
